# **24040208008-302(P)-AIII**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = CIFAR10(root="./train/", train=True, download=True, transform=transform)
test_data = CIFAR10(root="./test/", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)

classes = train_data.classes


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%")

print("Training Finished!")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"\nFinal Test Accuracy: {test_accuracy:.2f}%")


100%|██████████| 170M/170M [00:03<00:00, 48.4MB/s]
100%|██████████| 170M/170M [00:03<00:00, 46.4MB/s]


Epoch [1/10] - Loss: 1.3765 - Train Accuracy: 52.22%
Epoch [2/10] - Loss: 1.0933 - Train Accuracy: 62.05%
Epoch [3/10] - Loss: 0.9912 - Train Accuracy: 65.85%
Epoch [4/10] - Loss: 0.9273 - Train Accuracy: 68.19%
Epoch [5/10] - Loss: 0.8824 - Train Accuracy: 69.67%
Epoch [6/10] - Loss: 0.8455 - Train Accuracy: 70.89%
Epoch [7/10] - Loss: 0.8135 - Train Accuracy: 72.28%
Epoch [8/10] - Loss: 0.7856 - Train Accuracy: 73.13%
Epoch [9/10] - Loss: 0.7611 - Train Accuracy: 73.99%
Epoch [10/10] - Loss: 0.7335 - Train Accuracy: 74.75%
Training Finished!

Final Test Accuracy: 64.30%
